# 📊 Day 1-00.06: 모델 평가하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **학습된 모델** 로드하기
- **성능 평가**하기 (매우 간단하게!)
- **원본 모델과 비교**하기
- **결과 분석**하기

## 💡 모델 평가란?
**학습된 모델이 얼마나 잘 작동하는지 확인하는 것**

### 🔧 평가 방법들
1. **정성적 평가**: 사람이 직접 답변 품질 확인
2. **정량적 평가**: 수치로 성능 측정
3. **비교 평가**: 원본 모델과 성능 비교
4. **실제 사용**: 실제 질문에 답변해보기

### 📊 오늘 할 평가
- **질문 답변 품질**: 답변이 정확하고 유용한지
- **RAFT 성능**: 문서를 참고해서 답변하는지
- **한국어 이해**: 한국어 질문을 잘 이해하는지
- **일관성**: 같은 질문에 일관된 답변을 하는지


## 1. 필요한 라이브러리 불러오기


In [ ]:
# 모델 평가에 필요한 라이브러리들을 불러옵니다
import torch
import json
import time
import random
from transformers import (
    AutoTokenizer,           # 토크나이저
    AutoModelForCausalLM,    # 언어 모델
    BitsAndBytesConfig       # 4비트 양자화
)
from peft import (
    PeftModel,               # LoRA 모델 로드
    LoraConfig,              # LoRA 설정
    TaskType                 # 태스크 타입
)
import warnings
warnings.filterwarnings('ignore')

print("✅ 모델 평가 라이브러리가 준비되었습니다!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"🚀 CUDA 사용 가능: {torch.cuda.is_available()}")


## 2. 학습된 모델 로드하기


In [ ]:
# 학습된 모델을 로드합니다
print("📥 학습된 모델 로드 중...")

# 학습 설정 불러오기
with open("models/training_config.json", "r", encoding="utf-8") as f:
    training_config = json.load(f)

model_name = training_config['model_name']
print(f"   - 기본 모델: {model_name}")

# 4비트 양자화 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# 기본 모델 로드
print("🤖 기본 모델 로드 중...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# 토크나이저 로드
print("🔤 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# LoRA 모델 로드
print("🔧 LoRA 모델 로드 중...")
model = PeftModel.from_pretrained(base_model, "models/fine_tuned_lora")

print("✅ 학습된 모델 로드 완료!")
print(f"   - 기본 모델: {model_name}")
print(f"   - LoRA 어댑터: models/fine_tuned_lora/")
print(f"   - 디바이스: {next(model.parameters()).device}")

# 모델 정보 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"   - 학습 가능한 파라미터: {trainable_params:,}개")
print(f"   - 전체 파라미터: {all_params:,}개")
print(f"   - 학습 비율: {trainable_params/all_params*100:.2f}%")


## 3. 텍스트 생성 함수 만들기


In [ ]:
# 모델이 텍스트를 생성하는 함수를 만듭니다 (매우 간단하게!)
def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7):
    """
    모델이 텍스트를 생성하는 함수
    
    Args:
        model: 사용할 모델
        tokenizer: 토크나이저
        prompt: 입력 텍스트
        max_length: 최대 생성 길이
        temperature: 창의성 조절 (0.1=보수적, 1.0=창의적)
    
    Returns:
        생성된 텍스트
    """
    # 입력 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 입력 부분 제거하고 생성된 부분만 반환
    generated_part = generated_text[len(prompt):].strip()
    
    return generated_part

# 함수 테스트
print("🧪 텍스트 생성 함수 테스트:")
test_prompt = "질문: 인공지능이란 무엇인가요?\n답변:"
generated = generate_text(model, tokenizer, test_prompt, max_length=50)
print(f"   - 입력: {test_prompt}")
print(f"   - 생성: {generated}")
print("✅ 텍스트 생성 함수 준비 완료!")
